In [1]:
import sys
sys.path.append('/root/autodl-tmp/CommitFit')
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.combined_model as cm 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score, precision_recall_curve

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'A':0, 'C':1, 'P':2}
df = df.rename(columns={'maintenance_type':'label','msgs':'message','diffs':'diff'})
df = df.replace({"label": label2id})
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_3436/1967108509.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


,language,commit,commit_url,label,user,repo,feature,diff,message
0,C++,be937a3290223f926fe50684f1344569a573ed4b,https://github.com/tensorflow/tensorflow/commi...,1,tensorflow,tensorflow,"[1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 6, ...",diff --git a/tensorflow/compiler/mlir/lite/fla...,Fixing per axis quantization bug in flatbuffer...
1,C++,c800d2e36954edddcb83aa1df7f623f2780c7780,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/tensorflow/python/tools/saved_mod...,Add support for python expression as SavedMode...
2,C++,469e56eeff17eb857c95e935ca7b49723c43470e,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/.github/ISSUE_TEMPLATE/00-bug-iss...,Fix GitHub issue templates. Actual fix for #36...
3,C++,24f5d302c0906f3377f4d6bcc2735d943ce13e3d,https://github.com/tensorflow/tensorflow/commi...,0,tensorflow,tensorflow,"[3, 37, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/compiler/xla/pjrt/dist...,"Add new PjRT distributed APIs (KeyValueDelete,..."
4,C++,1effdb7b959503596b07f7f3e74618ab63f9e5e5,https://github.com/tensorflow/tensorflow/commi...,2,tensorflow,tensorflow,"[1, 0, 12, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/tensorflow/core/common_runtime/ea...,Remove IsLocalDevice() since it's no longer us...
...,...,...,...,...,...,...,...,...,...
1576,Java,e4bbfd537a63a52a6094491504fc7b0bb54bcb0e,,1,kiegroup,drools,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-persistence-jpa/src/main/j...,fixing reflection constructor to use int inste...
1577,Java,75b81dcbd26a11d5b370ea5bdf102c2d62ebd0c3,,1,kiegroup,drools,"[2, 6, 2, 3, 0, 0, 0, 1, 7, 0, 0, 0, 0, 0, 0, ...",diff --git a/drools-compiler/src/main/java/org...,Fix OutOfBoundException on- MemoryFileSystem.e...
1578,Java,6bc9677bbe919cd5bcadd8af2f5b0c838757a4ce,,1,kiegroup,drools,"[1, 1, 5, 13, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,[DROOLS-293] fix ObjectTypeNode id creation an...
1579,Java,48701edad8513b27acec7216581e64637157c86a,,0,kiegroup,drools,"[2, 24, 2, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/drools-core/src/main/java/org/dro...,DROOLS-515 Kie-Camel is broken after Camel Upd...


In [3]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [4]:
# train.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/COLARE-main/dataset/multi-lang-exp/eval.csv',index=False)
# train.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/train.csv',index=False)
# test.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/test.csv',index=False)
# val.to_csv('/root/autodl-tmp/CommitFit/dataset/multi-lang-exp/eval.csv',index=False)

In [5]:
# df_dataset[0]

In [6]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/bert-base-uncased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
train

,language,commit,commit_url,label,user,repo,feature,diff,message
1080,Rust,e0b52079daeec08e5d349d686e217c8ba11c1f64,https://github.com/bevyengine/bevy/commit/e0b5...,0,bevyengine,bevy,"[1, 17, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",diff --git a/crates/bevy_render/src/renderer/r...,Implement RenderResource for Box<T> (#1893)\n\...
590,JavaScript,ddff0f642acbf650b53d1ddffe544900569ab983,https://github.com/mrdoob/three.js/commit/ddff...,2,mrdoob,three.js,"[2, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/examples/js/SimulationRenderer.js...,Add comments to FF RGB format fix\n\n
679,JavaScript,57f2d7de58427a413c406ef0a5a4e120e3351620,https://github.com/chartjs/Chart.js/commit/57f...,2,chartjs,Chart.js,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/docs/00-Getting-Started.md b/docs...,Update docs and add plugin section (suggested ...
1100,Rust,b00e9fc227dc2ab1070326f29fbe9626702dfa67,https://github.com/helix-editor/helix/commit/b...,1,helix-editor,helix,"[2, 5, 0, 2, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/helix-term/src/commands.rs b/heli...,Handle line endings correctly in surround\n\n
1190,TypeScript,c168a92f187943a401f6c4095b5abde44478d0c9,https://github.com/ant-design/ant-design/commi...,2,ant-design,ant-design,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/components/tag/demo/control.md b/...,docs: add missing key prop for controlled tag ...
...,...,...,...,...,...,...,...,...,...
1130,TypeScript,defd45ccbadb7e0e347b1999d48fe354998ba37a,https://github.com/vuejs/vue/commit/defd45ccba...,2,vuejs,vue,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/src/compiler/optimizer.js b/src/c...,Typo fix: optimizier -> optimizer (#4015)\n\n\n
1294,TypeScript,bd34203bc40d6a406dfac3ab6cb41f5dfed63a77,https://github.com/vitejs/vite/commit/bd34203b...,1,vitejs,vite,"[1, 1, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/packages/vite/src/node/plugins/im...,fix: Avoid importing in source that __vitePrel...
860,Python,c3842edd8ea5da4626ffe6cb7f6c1965fa956757,https://github.com/keras-team/keras/commit/c38...,2,keras-team,keras,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",diff --git a/keras/engine/training.py b/keras/...,Corrected 3 small typos in training.py error m...
1459,Java,2947b932624e4931333c08cfd9c111628bbd25e9,,1,orientechnologies,orientdb,"[4, 5, 3, 3, 0, 0, 0, 3, 1, 0, 2, 0, 0, 0, 122...",diff --git a/client/src/main/java/com/orientec...,Fixed some errors in remote calls.--


In [8]:
train.reset_index(inplace=True)
val.reset_index(inplace=True)
test.reset_index(inplace=True)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
val_dataset = preprocesser.SentencePairDataset(val, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)



# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)
# for batch in train_loader:
#     print(batch)


base_model1 = cm.BaseModel(bert_model)
base_model2 = cm.BaseModel(codebert_model)

# Create stacking model
combined_model = cm.CombinedModel(base_model1, base_model2)
# Train the model
combined_model.trainer(train_loader, val_loader,num_epochs=1, patience=3)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/1 Loss: 0.7191: 100%|██████████| 139/139 [01:30<00:00,  1.53batch/s]


=============================train========================
Validation Accuracy: 0.7595
Precision: 0.7647
Recall: 0.7595
F1-Score: 0.7564
              precision    recall  f1-score   support

           0     0.6667    0.4706    0.5517        34
           1     0.6623    0.8500    0.7445        60
           2     0.8309    0.7902    0.8100       143

    accuracy                         0.7595       237
   macro avg     0.7200    0.7036    0.7021       237
weighted avg     0.7647    0.7595    0.7564       237



In [9]:
# test_dataset = preprocesser.SentencePairDataset(test, bert_tokenizer, codebert_tokenizer,message='message',command='diff',label='label')
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
0

In [10]:
test_acc, test_labels, test_probabilities, test_embeddings, test_predictions = combined_model.evaluate(test_loader)

Validation Accuracy: 0.7857
Precision: 0.8021
Recall: 0.7857
F1-Score: 0.7850
              precision    recall  f1-score   support

           0     0.7273    0.5000    0.5926        32
           1     0.6301    0.9020    0.7419        51
           2     0.8741    0.8065    0.8389       155

    accuracy                         0.7857       238
   macro avg     0.7438    0.7361    0.7245       238
weighted avg     0.8021    0.7857    0.7850       238



In [11]:
import torch

# 推荐：只保存参数字典
torch.save(combined_model.state_dict(), "combined_model_state.pth")